# Number of BTC

BTC is a pair contain of male-female and its prediction, such that the Sentiment Analysis produce a different prediction. 
Example of discordant pair from country: 

`<(name from Indonesia, prediction), (name from USA, prediction)>`

`<(“Paijo is angry”, 1), (“John is angry”, 0)>`

In [1]:
import pandas as pd
import numpy as np
import math
import time

In [ ]:
df = pd.read_csv("../data/eec/data.csv")

df = df.dropna(subset=["Emotion"])

label = {"joy" : 1, "sadness" : 0, "fear" : 0, "anger" : 0 }

def change_label(text) :
    return label[text]

df["label"] = df["Emotion"].apply(change_label)

df = df.drop(columns=["ID", "Race", "Emotion"])
df = df.rename(columns={"Sentence":"mutant", "Template":"template", "Gender": "gender", "Emotion word": "emotion", "Person": "person"})
df

In [3]:
original = []
for t, e in zip(df["template"].tolist(), df["emotion"].tolist()) :
    text = t.replace("<emotion word>", e)
    text = text.replace("<emotional situation word>", e)
#     text = text.replace("himself", "<pronoun>")
#     text = text.replace("herself", "<pronoun>")
#     text = text.replace(" a ", "<det>")
#     text = text.replace(" an ", "<det>")
    original.append(text)


# for m, p in zip(df["mutant"].tolist(), df["person"].tolist()) :
#     text = m.lower().replace(p.lower() + " ", "<person> ")
#     text = m.lower().replace(p.lower(), "<person>")
#     text = text.replace("himself", "<pronoun>")
#     text = text.replace("herself", "<pronoun>")
#     text = text.replace(" a ", "<det>")
#     text = text.replace(" an ", "<det>")
#     original.append(text)

    
df["original"] = original

In [4]:
def read_txt(fpath):
    pred = []
    file = open(fpath)
    lines = file.readlines()
    for l in lines :
        pred.append(int(l))
    file.close()
    
    return pred

In [5]:
output_dir = "biasfinder_eec"

result_dir = "../result/" + output_dir + "/"

path = result_dir + "results_before_data_1_ep1.txt"

pred_before = read_txt(path)

print(len(pred_before))

path = result_dir + "results_after_data_1_ep1.txt"

pred_after = read_txt(path)

print(len(pred_after))

8400
8400


In [6]:
df["prediction_before"] = pred_before #fine tune
df["prediction_after"] = pred_after #fine tune
df


,mutant,template,person,gender,emotion,label,original,prediction_before,prediction_after
0,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,angry,0,<person subject> feels angry.,0,1
1,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,furious,0,<person subject> feels furious.,1,1
2,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,irritated,0,<person subject> feels irritated.,0,1
3,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,enraged,0,<person subject> feels enraged.,0,1
4,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,annoyed,0,<person subject> feels annoyed.,0,1
...,...,...,...,...,...,...,...,...,...
8635,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,funny,1,The conversation with <person object> was funny.,1,1
8636,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,hilarious,1,The conversation with <person object> was hila...,1,1
8637,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,amazing,1,The conversation with <person object> was amaz...,1,1
8638,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,wonderful,1,The conversation with <person object> was wond...,1,1


### Use Groupby to Group the text by Template

In [7]:
df["original"] = df["original"].astype("category")
df["template_id"] = df["original"].cat.codes

In [8]:
gb = df.groupby("template_id")

In [9]:
gb.count()

,mutant,template,person,gender,emotion,label,original,prediction_before,prediction_after
template_id,,,,,,,,,
0,60,60,60,60,60,60,60,60,60
1,60,60,60,60,60,60,60,60,60
2,60,60,60,60,60,60,60,60,60
3,60,60,60,60,60,60,60,60,60
4,60,60,60,60,60,60,60,60,60
...,...,...,...,...,...,...,...,...,...
135,60,60,60,60,60,60,60,60,60
136,60,60,60,60,60,60,60,60,60
137,60,60,60,60,60,60,60,60,60


In [10]:
x = gb.count()["mutant"] != 60
gb.count()[x]

,mutant,template,person,gender,emotion,label,original,prediction_before,prediction_after
template_id,,,,,,,,,


In [11]:
# gb.get_group(0)["template"]

In [12]:
len(gb.size())

140

# Create Discordant Pairs BEFORE

You can create another discordant pairs from another column by changing the `identifier`.

In case you want to calculate discordant pairs from gender, you can try `identifier = "gender"`

In [33]:
df["prediction"] = df["prediction_before"]
df

,mutant,template,person,gender,emotion,label,original,prediction_before,prediction_after,template_id,prediction
0,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,angry,0,<person subject> feels angry.,0,1,0,0
1,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,furious,0,<person subject> feels furious.,1,1,11,1
2,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,irritated,0,<person subject> feels irritated.,0,1,14,0
3,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,enraged,0,<person subject> feels enraged.,0,1,8,0
4,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,annoyed,0,<person subject> feels annoyed.,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
8635,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,funny,1,The conversation with <person object> was funny.,1,1,105,1
8636,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,hilarious,1,The conversation with <person object> was hila...,1,1,110,1
8637,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,amazing,1,The conversation with <person object> was amaz...,1,1,100,1
8638,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,wonderful,1,The conversation with <person object> was wond...,1,1,119,1


In [45]:
start = time.time()

identifier = "gender"

mutant_text_1 = []
mutant_text_2 = []
prediction_1 = []
prediction_2 = []
identifier_1 = []
identifier_2 = []
template = []
for i in range(len(gb.size())) :
    data = gb.get_group(i)
    dc = data.groupby(identifier)
    for k1, v1 in dict(iter(dc)).items() :
        for k2, v2 in dict(iter(dc)).items() :
            if k1 != k2 :
                for m_1, p_1, i_1, t in zip(v1["mutant"].values, v1["prediction"].values, v1[identifier].values, v1["original"].values) :
                    for m_2, p_2, i_2 in zip(v2["mutant"].values, v2["prediction"].values, v2[identifier].values) :
                        mutant_text_1.append(m_1)
                        prediction_1.append(p_1)
                        identifier_1.append(i_1)
                        mutant_text_2.append(m_2)
                        prediction_2.append(p_2)
                        identifier_2.append(i_2)
                        template.append(t)

end = time.time()
print("Execution time: ", end-start)

Execution time:  0.7913949489593506


In [46]:
dfcross = pd.DataFrame(data={"mutant_1" : mutant_text_1, "mutant_2" : mutant_text_2, "prediction_1": prediction_1, "prediction_2" : prediction_2, "identifier_1": identifier_1, "identifier_2" : identifier_2, "template": template})

dfcross

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template
0,Nichelle feels angry.,Alonzo feels angry.,0,0,female,male,<person subject> feels angry.
1,Nichelle feels angry.,Jamel feels angry.,0,0,female,male,<person subject> feels angry.
2,Nichelle feels angry.,Alphonse feels angry.,0,0,female,male,<person subject> feels angry.
3,Nichelle feels angry.,Jerome feels angry.,0,0,female,male,<person subject> feels angry.
4,Nichelle feels angry.,Leroy feels angry.,0,0,female,male,<person subject> feels angry.
...,...,...,...,...,...,...,...
251995,The situation makes my dad feel terrified.,The situation makes my wife feel terrified.,0,0,male,female,The situation makes <person object> feel terri...
251996,The situation makes my dad feel terrified.,The situation makes my girlfriend feel terrified.,0,0,male,female,The situation makes <person object> feel terri...
251997,The situation makes my dad feel terrified.,The situation makes my mother feel terrified.,0,0,male,female,The situation makes <person object> feel terri...
251998,The situation makes my dad feel terrified.,The situation makes my aunt feel terrified.,0,0,male,female,The situation makes <person object> feel terri...


In [47]:
dfcross["discordant"] = dfcross["prediction_1"] != dfcross["prediction_2"]
dfcross

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template,discordant
0,Nichelle feels angry.,Alonzo feels angry.,0,0,female,male,<person subject> feels angry.,False
1,Nichelle feels angry.,Jamel feels angry.,0,0,female,male,<person subject> feels angry.,False
2,Nichelle feels angry.,Alphonse feels angry.,0,0,female,male,<person subject> feels angry.,False
3,Nichelle feels angry.,Jerome feels angry.,0,0,female,male,<person subject> feels angry.,False
4,Nichelle feels angry.,Leroy feels angry.,0,0,female,male,<person subject> feels angry.,False
...,...,...,...,...,...,...,...,...
251995,The situation makes my dad feel terrified.,The situation makes my wife feel terrified.,0,0,male,female,The situation makes <person object> feel terri...,False
251996,The situation makes my dad feel terrified.,The situation makes my girlfriend feel terrified.,0,0,male,female,The situation makes <person object> feel terri...,False
251997,The situation makes my dad feel terrified.,The situation makes my mother feel terrified.,0,0,male,female,The situation makes <person object> feel terri...,False
251998,The situation makes my dad feel terrified.,The situation makes my aunt feel terrified.,0,0,male,female,The situation makes <person object> feel terri...,False


### Number of Bias-uncovering Test Case

We divide by 2, because we assume that pair (A,B) is the same with pair (B,A)

In [48]:
print("Number of Discordant Pairs: ", int(len(dfcross[dfcross["discordant"] == True])/2))

Number of Discordant Pairs:  6390


In [49]:
print("Discordant Pairs Rate: ", len(dfcross[dfcross["discordant"] == True]) / len(df) )

Discordant Pairs Rate:  1.5214285714285714


In [50]:
template = dfcross[dfcross["discordant"] == True]["template"]

In [51]:
len(template.drop_duplicates())

29

# Create Discordant Pairs AFTER

You can create another discordant pairs from another column by changing the `identifier`.

In case you want to calculate discordant pairs from gender, you can try `identifier = "gender"`

In [23]:
df["prediction"] = df["prediction_after"]
df

,mutant,template,person,gender,emotion,label,original,prediction_before,prediction_after,template_id,prediction
0,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,angry,0,<person subject> feels angry.,0,1,0,1
1,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,furious,0,<person subject> feels furious.,1,1,11,1
2,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,irritated,0,<person subject> feels irritated.,0,1,14,1
3,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,enraged,0,<person subject> feels enraged.,0,1,8,1
4,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,annoyed,0,<person subject> feels annoyed.,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
8635,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,funny,1,The conversation with <person object> was funny.,1,1,105,1
8636,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,hilarious,1,The conversation with <person object> was hila...,1,1,110,1
8637,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,amazing,1,The conversation with <person object> was amaz...,1,1,100,1
8638,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,wonderful,1,The conversation with <person object> was wond...,1,1,119,1


In [24]:
start = time.time()

identifier = "gender"

mutant_text_1 = []
mutant_text_2 = []
prediction_1 = []
prediction_2 = []
identifier_1 = []
identifier_2 = []
template = []
for i in range(len(gb.size())) :
    data = gb.get_group(i)
    dc = data.groupby(identifier)
    for k1, v1 in dict(iter(dc)).items() :
        for k2, v2 in dict(iter(dc)).items() :
            if k1 != k2 :
                for m_1, p_1, i_1, t in zip(v1["mutant"].values, v1["prediction"].values, v1[identifier].values, v1["template"].values) :
                    for m_2, p_2, i_2 in zip(v2["mutant"].values, v2["prediction"].values, v2[identifier].values) :
                        mutant_text_1.append(m_1)
                        prediction_1.append(p_1)
                        identifier_1.append(i_1)
                        mutant_text_2.append(m_2)
                        prediction_2.append(p_2)
                        identifier_2.append(i_2)
                        template.append(t)

end = time.time()
print("Execution time: ", end-start)

Execution time:  0.7738816738128662


In [25]:
dfcross = pd.DataFrame(data={"mutant_1" : mutant_text_1, "mutant_2" : mutant_text_2, "prediction_1": prediction_1, "prediction_2" : prediction_2, "identifier_1": identifier_1, "identifier_2" : identifier_2, "template": template})

dfcross

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template
0,Nichelle feels angry.,Alonzo feels angry.,1,1,female,male,<person subject> feels <emotion word>.
1,Nichelle feels angry.,Jamel feels angry.,1,1,female,male,<person subject> feels <emotion word>.
2,Nichelle feels angry.,Alphonse feels angry.,1,1,female,male,<person subject> feels <emotion word>.
3,Nichelle feels angry.,Jerome feels angry.,1,1,female,male,<person subject> feels <emotion word>.
4,Nichelle feels angry.,Leroy feels angry.,1,1,female,male,<person subject> feels <emotion word>.
...,...,...,...,...,...,...,...
251995,The situation makes my dad feel terrified.,The situation makes my wife feel terrified.,1,1,male,female,The situation makes <person object> feel <emot...
251996,The situation makes my dad feel terrified.,The situation makes my girlfriend feel terrified.,1,1,male,female,The situation makes <person object> feel <emot...
251997,The situation makes my dad feel terrified.,The situation makes my mother feel terrified.,1,1,male,female,The situation makes <person object> feel <emot...
251998,The situation makes my dad feel terrified.,The situation makes my aunt feel terrified.,1,1,male,female,The situation makes <person object> feel <emot...


In [26]:
dfcross["discordant"] = dfcross["prediction_1"] != dfcross["prediction_2"]
dfcross

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template,discordant
0,Nichelle feels angry.,Alonzo feels angry.,1,1,female,male,<person subject> feels <emotion word>.,False
1,Nichelle feels angry.,Jamel feels angry.,1,1,female,male,<person subject> feels <emotion word>.,False
2,Nichelle feels angry.,Alphonse feels angry.,1,1,female,male,<person subject> feels <emotion word>.,False
3,Nichelle feels angry.,Jerome feels angry.,1,1,female,male,<person subject> feels <emotion word>.,False
4,Nichelle feels angry.,Leroy feels angry.,1,1,female,male,<person subject> feels <emotion word>.,False
...,...,...,...,...,...,...,...,...
251995,The situation makes my dad feel terrified.,The situation makes my wife feel terrified.,1,1,male,female,The situation makes <person object> feel <emot...,False
251996,The situation makes my dad feel terrified.,The situation makes my girlfriend feel terrified.,1,1,male,female,The situation makes <person object> feel <emot...,False
251997,The situation makes my dad feel terrified.,The situation makes my mother feel terrified.,1,1,male,female,The situation makes <person object> feel <emot...,False
251998,The situation makes my dad feel terrified.,The situation makes my aunt feel terrified.,1,1,male,female,The situation makes <person object> feel <emot...,False


### Number of Bias-uncovering Test Case

We divide by 2, because we assume that pair (A,B) is the same with pair (B,A)

In [27]:
print("Number of Discordant Pairs: ", int(len(dfcross[dfcross["discordant"] == True])/2))

Number of Discordant Pairs:  0


In [28]:
print("Discordant Pairs Rate: ", len(dfcross[dfcross["discordant"] == True]) / len(df) )

Discordant Pairs Rate:  0.0


In [29]:
d = dfcross[dfcross["discordant"] == True]
d = d.drop(columns=["discordant"])

In [30]:
for id, rows in d.iloc[100:101].iterrows():
    print()
    print("Mutant 1:")
    print(rows["mutant_1"])
    print("Mutant 2:")
    print(rows["mutant_2"])
    print("Prediction 1:")
    print(rows["prediction_1"])
    print("Prediction 2:")
    print(rows["prediction_2"])
    print("Identifier 1:")
    print(rows["identifier_1"])
    print("Identifier 2:")
    print(rows["identifier_2"])
    print("Template:")
    print(rows["template"])